In [1]:
'''
the purpose of this notebook is to find any features that are very similar to the features that were hits.
the colinear stuff could be other possible effectors

The current finding is that there are no features with greater than 95% colinarity with any of the three hits.
i.e. cosine_similarity > 0.95
though PMT has two with cosine_distance > 0.9, with the two subsequent blocks. The second block contains RAD5.. which
might be what the other studies have been finding. But this could also work the other way around. For both of these blocks,
subpopoulation 1 (0-index) correlated highly with PMT's subpopulation 1, and was probably dropped from the analysis.
It seems like this sort of correlation analysis is quite necessary..
'''



"\nthe purpose of this notebook is to find any features that are very similar to the features that were hits.\nthe colinear stuff could be other possible effectors\n\nThe current finding is that there are no features with greater than 95% colinarity with any of the three hits.\ni.e. cosine_similarity > 0.95\nthough PMT has two with cosine_distance > 0.9, with the two subsequent blocks. The second block contains RAD5.. which\nmight be what the other studies have been finding. But this could also work the other way around. For both of these blocks,\nsubpopoulation 1 (0-index) correlated highly with PMT's subpopulation 1, and was probably dropped from the analysis.\nIt seems like this sort of correlation analysis is quite necessary..\n"

In [2]:
import numpy as np, pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from pathlib import Path
directory = Path('/d/data/plasmo/nn_scalar3')
# directory = Path('/d/data/neis/neis_nn')

In [3]:
data_file_target = directory / 'output_mod' / 'data.tsv'
# data_file_target = directory / 'output' /  'data.tsv'
target_df = np.loadtxt(data_file_target, delimiter=' ')
important_idx = [289]
targets = target_df.T[important_idx]

In [4]:
data_file_full = directory / 'nat_v3_nn.tsv'
# data_file_full = directory / 'neis_nn.tsv'
full_df = pd.read_csv(data_file_full, sep='\t', header=0, index_col=0)
meta_file = directory / 'meta_mod.tsv'
# meta_file = directory / ''
meta_df = pd.read_csv(meta_file, sep='\t', header=0, index_col=0)
full_df, meta_df = full_df.align(meta_df, axis=0, join='inner')
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(full_df.values).T

/home/javi/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [5]:
def revOneHot(input, good_idx):
    n_vec = input.max(axis=0) + 1
    res = np.full(int(n_vec[0]), 0)
    for i, n in enumerate(n_vec[1:]):
        res = np.concatenate((res, np.full(int(n), i+1)))
    return res[good_idx]

In [6]:
similarity = cosine_similarity(encoded_data, targets)
np.sum((similarity>0.90), axis=0)

array([2])

In [11]:
good_idx = np.where((similarity > 0.9))
rev_idx = revOneHot(full_df.values, good_idx[0])

In [12]:
full_df.columns[rev_idx]

Index(['Pf3D7_14_v3:2240000', 'Pf3D7_14_v3:2245000'], dtype='object')

In [ ]:
np.sum(encoded_data[good_idx[0]], axis=1)

In [ ]:
revOneHot(full_df.values, [good_idx[0][-2]-2])

In [ ]:
good_idx[0][-1]